In [ ]:
import pickle
import torch
torch_dataset = torch.load('lastfm.pkl')

In [4]:
new_dataset = []
block_size = 512

for data in torch_dataset['sequences']:
    num_items = len(data['arrival_times'])
    for start_idx in range(0, num_items, block_size):
        seq = []
        for i in range(start_idx, min(start_idx + block_size, num_items)):
            elem = {
                'type_event': data['marks'][i],
                'time_since_start': data['arrival_times'][i],
                'time_since_last_event': (
                    data['arrival_times'][i] - data['arrival_times'][i-1]
                    if i > start_idx
                    else 0
                ),
            }
            seq.append(elem)
        new_dataset.append(seq)

In [5]:
import random
random.seed(2024)
random.shuffle(new_dataset)

alpha1 = 0.7
alpha2 = 0.85
ind1 = round(len(new_dataset) * alpha1)
ind2 = round(len(new_dataset) * alpha2)

train_dataset = {
    'dim_process': torch_dataset['num_marks'],
    'train': new_dataset[:ind1]
}

dev_dataset = {
    'dim_process': torch_dataset['num_marks'],
    'dev': new_dataset[ind1:ind2]
}

test_dataset = {
    'dim_process': torch_dataset['num_marks'],
    'test': new_dataset[ind2:]
}

In [6]:
import pickle

with open(f"train.pkl", "wb") as f:
    pickle.dump(train_dataset, f)
with open(f"dev.pkl", "wb") as f:
    pickle.dump(dev_dataset, f)
with open(f"test.pkl", "wb") as f:
    pickle.dump(test_dataset, f)